### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [32]:
### Open AI API key and open source model --> Llama3, Gemma2, mistral 
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [33]:
from langchain_groq import ChatGroq

model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002887B558A10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002887CD80150>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [34]:
from langchain_core.messages import HumanMessage,SystemMessage

message=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello how are you?")
]

In [35]:
result=model.invoke(message)

In [36]:
result

AIMessage(content='The most common way to say "Hello, how are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **comment:** how\n* **allez-vous:** are you (formal)\n\n\nLet me know if you\'d like to know other ways to say it!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 21, 'total_tokens': 100, 'completion_time': 0.143636364, 'prompt_time': 0.001320741, 'queue_time': 0.250116549, 'total_time': 0.144957105}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c94a2a35-8759-404b-8a67-e04e5b2dbb08-0', usage_metadata={'input_tokens': 21, 'output_tokens': 79, 'total_tokens': 100})

In [37]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'The most common way to say "Hello, how are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **comment:** how\n* **allez-vous:** are you (formal)\n\n\nLet me know if you\'d like to know other ways to say it!\n'

In [38]:
### using LCEL - chain the component
chain= model|parser
chain.invoke(message)

'The most common way to say "Hello, how are you?" in French is:\n\n**Bonjour, comment allez-vous ?**\n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **comment:** how\n* **allez-vous:** are you (formal, plural) \n\n\nLet me know if you\'d like other ways to say it! \n'

In [39]:
### prompt template
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [40]:
result=prompt.invoke({"language":"french","text":"hello"})

In [41]:
result.to_messages()

[SystemMessage(content='Translate the following into french:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [42]:
chain=prompt|model|parser
chain.invoke({"language":"french","text":"how are you"})

'The most common way to say "how are you" in French is:\n\n**Comment allez-vous ?** (formal)\n\n**Comment vas-tu ?** (informal)\n\n\nYou would use "Comment allez-vous?" with someone you don\'t know well, someone older, or someone in a position of authority.  "Comment vas-tu?" is used with friends, family, and people your own age. \n'